In [1]:
!sudo apt-get install -y build-essential libasound2-dev libjack-dev portaudio19-dev




build-essential is already the newest version (12.8ubuntu1.1).
The following additional packages will be installed:
  libjack0 libportaudio2 libportaudiocpp0 uuid-dev
Suggested packages:
  libasound2-doc jackd1 portaudio19-doc
The following packages will be REMOVED:
  libjack-jackd2-0
The following NEW packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
  portaudio19-dev uuid-dev
0 upgraded, 7 newly installed, 1 to remove and 76 not upgraded.
Need to get 625 kB of archives.
After this operation, 3340 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libjack0 amd64 1:0.125.0-3build2 [93.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libasound2-dev amd64 1.2.2-2.1ubuntu2.5 [104 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 uuid-dev amd64 2.34-0.1ubuntu9.3 [33.6 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libjack-dev amd64 1:0

In [2]:
# !rm -r /kaggle/working/magenta
!cp -r /kaggle/input/mag-git-bundleedit/magenta1/magenta /kaggle/working/

In [3]:
!cd /kaggle/working/magenta && pip install -e . --ignore-installed rtmidi

Obtaining file:///kaggle/working/magenta
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 

In [4]:
# sequence = convert_musicxml(root_dir, sub_dir, full_file_path)

from magenta.scripts.convert_dir_to_note_sequences import convert_musicxml
from note_seq import split_note_sequence
import tensorflow.compat.v1 as tf
import os

prepath = "/kaggle/input/bachxml/xml"
all_ns = []
count = 0
for file in os.listdir(prepath):
#     print(file)
    try:
        ns = convert_musicxml(prepath, "xml", os.path.join(prepath, file))

    except: 
        print("Error on:", file)
        count += 1
        continue
    
    ns_split = split_note_sequence(ns,
                            20,
                            False)
#     print(len(ns_split))
    count += 1
    all_ns += ns_split
    if count%100 == 0:
        print(count)

print(len(all_ns))
!mkdir /kaggle/tmp/
if (os.path.exists("/kaggle/tmp/bach.tfrecord") == False):
    f = open("/kaggle/tmp/bach.tfrecord", "w")
with tf.io.TFRecordWriter("/kaggle/tmp/bach.tfrecord") as writer:
    for sequence in all_ns:
        writer.write(sequence.SerializeToString())

/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


100
200
300
400
880


In [5]:
!ls /kaggle/tmp/

bach.tfrecord


In [6]:
tf.logging.set_verbosity(tf.logging.INFO)
from magenta.pipelines import pipeline, note_sequence_pipelines, statistics
from note_seq.protobuf import music_pb2
from magenta.models import polyphony_rnn
from magenta.models.polyphony_rnn import polyphony_rnn_pipeline
from magenta.models.polyphony_rnn import polyphony_model
pipeline_instance = polyphony_rnn.polyphony_rnn_pipeline.get_pipeline(
    min_steps=80,  # 5 measures
    max_steps = None,
    eval_ratio=0.2,
    config=polyphony_model.default_configs['polyphony'])

input_dir = '/kaggle/tmp/bach.tfrecord'
output_dir = '/kaggle/tmp/bachrecords'
pipeline.run_pipeline_serial(
  pipeline_instance,
  pipeline.tf_record_iterator(input_dir, pipeline_instance.input_type),
  output_dir)


In [7]:
print(sum(1 for _ in tf.python_io.tf_record_iterator('/kaggle/tmp/bachrecords/eval_poly_tracks.tfrecord')),
sum(1 for _ in tf.python_io.tf_record_iterator('/kaggle/tmp/bachrecords/training_poly_tracks.tfrecord')))

1152 4725


In [8]:
from magenta import common
from magenta.models.shared import events_rnn_graph, events_rnn_train
config = polyphony_model.default_configs['polyphony']
config.hparams.parse("learning_rate=0.005")
mode = 'train' 
#     if FLAGS.eval else 'train'
train_record = tf.gfile.Glob(
    os.path.expanduser('/kaggle/tmp/bachrecords/training_poly_tracks.tfrecord'))
build_graph_fn_train = events_rnn_graph.get_build_graph_fn(
    mode, config, train_record)
run_dir = os.path.expanduser('/kaggle/working/bachruns/logdir/run1')
train_dir = os.path.join(run_dir, 'train')
tf.gfile.MakeDirs(train_dir) 
# bund = os.path.expanduser('/kaggle/working/logdir/trained.mag')
events_rnn_train.run_training(build_graph_fn_train, train_dir,
                              2000,
                              summary_frequency = 10,
                              checkpoints_to_keep=200)

# EVAL 
mode = 'eval'
eval_record = tf.gfile.Glob(os.path.expanduser('/kaggle/tmp/bachrecords/eval_poly_tracks.tfrecord'))
batch_count = (common.count_records(eval_record) / config.hparams.batch_size)
build_graph_fn_eval = events_rnn_graph.get_build_graph_fn(
    mode, config, eval_record)
eval_dir = os.path.join(run_dir, 'eval')
tf.gfile.MakeDirs(eval_dir)
events_rnn_train.run_eval(build_graph_fn_eval, train_dir, eval_dir, num_batches=batch_count,
             timeout_secs=300)

/kaggle/working/magenta/models/shared/events_rnn_graph.py:50: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = base_cell(rnn_layer_sizes[i])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
